In [20]:
import os
import cv2


In [27]:

path_to_imgs = "/Users/fedor.denisov/Development/tasks/eggs_vs_pan_recognition/dataset/train/images/"
path_to_masks = "/Users/fedor.denisov/Development/tasks/eggs_vs_pan_recognition/dataset/train/masks/"

path_to_formatted_dataset = "/Users/fedor.denisov/Development/tasks/eggs_vs_pan_recognition/dataset_train_formatted_4/"

path_train_imgs = path_to_formatted_dataset + "train/"
path_train_masks = path_to_formatted_dataset + "train_labels/"
path_validation_imgs = path_to_formatted_dataset + "val/"
path_validation_masks = path_to_formatted_dataset + "val_labels/"
path_dev_imgs = path_to_formatted_dataset + "test/"
path_dev_masks = path_to_formatted_dataset + "test_labels/"



In [28]:

def image_resize(image, width, height, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized


def resize_with_padding(image, size):
    h, w = image.shape[:2]
    if h/size[0] == w/size[1]:
        return image_resize(image, None, size[0])

    if h/size[0] > w/size[1]:
        resized = image_resize(image, None, size[1])
    else:
        resized = image_resize(image, size[0], None)

    h_res, w_res = resized.shape[:2]

    return cv2.copyMakeBorder(resized, 0, size[0] - h_res, 0, size[1] - w_res, cv2.BORDER_CONSTANT, value=[0, 0, 0])

def copy_image(path_to_imgs, img_name, new_path_to_imgs):
    original_path = path_to_imgs + img_name
    new_path = new_path_to_imgs + os.path.splitext(img_name)[0] + '.png'
    img = cv2.imread(original_path)
    image_padded = resize_with_padding(img,(512,512))
    cv2.imwrite(new_path, image_padded)
    

def validate_image_and_labels(path_to_imgs, path_to_labels, img_name):
    img = cv2.imread(path_to_imgs + img_name)
    label = cv2.imread(path_to_labels + img_name)
    return not (img is None or label is None)
    


In [29]:
images_filenames = os.listdir(path_to_imgs)


failed = 0
for image_filename in images_filenames:
    if not os.path.exists(path_to_imgs + image_filename):
        failed += 1
    if not os.path.exists(path_to_masks + image_filename):
        failed += 1

print(len(images_filenames))
print(failed)


398
0


In [30]:
from random import shuffle

images_filenames_available = list(filter(lambda image_name: 
                                         validate_image_and_labels(path_to_imgs, path_to_masks, image_name), 
                                         images_filenames))

num_dev = 70
num_validation = 28
num_train = len(images_filenames_available) - num_dev - num_validation

shuffle(images_filenames)


In [31]:

print ('start preprocessing')
for i in range(0,num_dev):
    copy_image(path_to_imgs, images_filenames[i], path_dev_imgs)
    copy_image(path_to_masks, images_filenames[i], path_dev_masks)
    
for i in range(num_dev, num_dev + num_validation):
    copy_image(path_to_imgs, images_filenames[i], path_validation_imgs)
    copy_image(path_to_masks, images_filenames[i], path_validation_masks)
    
    
for i in range(num_dev + num_validation, num_dev + num_validation + num_train):
    copy_image(path_to_imgs, images_filenames[i], path_train_imgs)
    copy_image(path_to_masks, images_filenames[i], path_train_masks)

print ('finish preprocessing')

start preprocessing
finish preprocessing


In [32]:



path_to_img_test = "/Users/fedor.denisov/Development/tasks/eggs_vs_pan_recognition/dataset_train_formatted_3/test/image_35.png"

img = cv2.imread(path_to_img_test)
    
image_padded = resize_with_padding(img,(1024,1024))

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
image_padded.shape

In [34]:

path_to_imgs = "/Users/fedor.denisov/Development/tasks/eggs_vs_pan_recognition/dataset/test/images/"
path_to_result = "/Users/fedor.denisov/Development/tasks/eggs_vs_pan_recognition/test_wl/images/"

images_filenames = os.listdir(path_to_imgs)

for filename in images_filenames:
    copy_image(path_to_imgs, filename, path_to_result)
    
    
    

In [ ]:
path_to_test_imgs = "/Users/fedor.denisov/Development/tasks/eggs_vs_pan_recognition/test_wl/images/"
path_to_masks = "/Users/fedor.denisov/Development/tasks/eggs_vs_pan_recognition/test_wl_result/masks/"
path_to_masks_cropped = "/Users/fedor.denisov/Development/tasks/eggs_vs_pan_recognition/test_wl_result/masks_cropped/"

images_filenames = os.listdir(path_to_test_imgs)

for filename in images_filenames:
    original_path_img = path_to_test_imgs + filename
    original_path_mask = path_to_masks + filename
    new_path = path_to_masks_cropped + os.path.splitext(filename)[0] + '.png'
    img = cv2.imread(original_path_img)
    mask = cv2.imread(original_path_mask)
    h, w = img.shape[:2]
    cv2.imwrite(new_path, mask[0:h,0:w])


